In [3]:
import pandas as pd

ALL_CSV_FILES:list[str] = list()    # should contain the relative path to all data files

df = pd.read_csv("kp_test/ALIPY_RANDOM/Iris/accuracy.csv.xz")   # read in the compressed csv file trough pandas library
df.head()           # prints the first entries of the data frames

'''
1. Lese die EXP_UNIQUE_ID aus und finde die korrespondierende Hyperparamerterkonfiguration in 05_done_workloads.csv
2. Nehme nur diejenigen Reihen auf, welche eine EXP_BATCH_SIZE von 5 haben (es gilt: EXP_BATCH_SIZE in [1,5,10])
3. Prüfe ob alle selektierten Dataframes nur Werte != NaN beinhalten, wenn nicht, interpoliere die zu füllenden Einträge
   auf Grundlage unserer gewähten Interpolationsstrategie (TODO)
4. Führe mit allen Datenvektoren PCA durch um eine Dimensionsreduktion zu erzielen (beschleunigt Clustering) 
   -> Dimensionen sollten danach für alle Datenvektoren gleich sein! 
5. Führe ein k-means Clustering durch, experimentiere dabei mit verschieden großen k um "bestes" Clustering zu finden
6. Führe ein Clustering durch, welches OHNE vorherige Initialisierung der Clusteranzahl und ähnliches auskommt (TODO: Recherche welche
   Strategie)
'''

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,EXP_UNIQUE_ID
0,0.950000,0.666667,0.666667,0.983333,1.000000,0.95,0.950000,1.000000,0.966667,0.950000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,50
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61
2,0.950000,0.783333,0.800000,0.800000,0.800000,1.00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,66
3,0.783333,0.833333,1.000000,1.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83
4,0.866667,0.583333,0.866667,0.850000,0.866667,0.95,0.966667,0.966667,0.966667,0.983333,...,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,0.983333,149
